<a target="_blank" href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/n4_bias_field_correct.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# N4 Bias field correction

The example below demonstrates extraction of radiomics features from CT scan and tumor segmentation for a batch of DICOM datasets.

### Install pyCERR

In [ ]:
%%capture
!pip install -U git+https://github.com/cerr/pyCERR/

#### Import the required modules

In [1]:
from cerr import plan_container as pc
from cerr.dataclasses import scan as scn
import SimpleITK as sitk
import numpy as np

#### Load data into planC container

In [2]:
# Load dicom to planC container
dcm_scan_dir = r"path/to/mr_dicom_dir"
planC = pc.load_dcm_dir(dcm_scan_dir)

('PURE-01_001-012', 'PURE-01_001-012', '1.3.12.2.1107.5.99.2.80500.30000021030111595013200277326', '1.2.826.0.1.3680043.8.498.27841218035895015762633437158085384576', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT', 'RTSTRUCT')
('PURE-01_001-012', 'PURE-01_001-012', '1.3.12.2.1107.5.99.2.80500.30000021030111595013200277326', '1.3.12.2.1107.5.99.2.80500.30000021030111595013200274239', 'MR', 'MR', 0.0, 0.0, '1', '1', '1')


#### Apply N4BiasFieldCorrectionImageFilter from SimpleITK

In [3]:
# Apply N4 bias field correction filter
scanNum = 0
scanITKObj = planC.scan[scanNum].getSitkImage()
corrector = sitk.N4BiasFieldCorrectionImageFilter()
corrector.SetMaximumNumberOfIterations([50, 50, 50, 50])
outImg = corrector.Execute(scanITKObj)
logBiasFieldImg = corrector.GetLogBiasFieldAsImage(scanITKObj)
logBiasField3M = scn.getCERRScanArrayFromITK(logBiasFieldImg, scanNum, planC)
correctedImageFullResolution = planC.scan[scanNum].getScanArray() / np.exp(logBiasField3M)

#### Import the N4 corrected image into planC

In [4]:
# Add bias field corrected array to planC
x, y, z = planC.scan[scanNum].getScanXYZVals()
planC = pc.import_scan_array(correctedImageFullResolution,x,y,z,'N4 Corrected',scanNum, planC)

### Visualize the original and N4 bias field corrected images

In [5]:
# Visualize original and bias field corrected images
from cerr import viewer as vwr
scanNum = [0, 1]
doseNum = []
strNum = []
displayMode = '2d'
viewer, scan_layer, dose_layer, struct_layer = \
        vwr.show_scan_struct_dose(scanNum, strNum, doseNum, planC, displayMode)